In [ ]:
import requests
import os
import openai
import pandas as pd
from concurrent.futures import ThreadPoolExecutor, ProcessPoolExecutor
from tenacity import retry, stop_after_attempt, wait_fixed


openai.api_type = "azure"
openai.api_version = "2023-07-01-preview"
openai.api_key = "YOURAPIKEY" 
engine = "GPT35"

from tenacity import (
    retry,
    stop_after_attempt,
    wait_chain,
    wait_fixed
) 

@retry(wait=wait_chain(*[wait_fixed(3) for i in range(3)] +
                       [wait_fixed(5) for i in range(2)] +
                       [wait_fixed(10)]))
def completion_with_backoff(**kwargs):
    return openai.ChatCompletion.create(**kwargs)


def verify_entity_exists(response, list_in_txt):
    system_prompt = "You are an AI assistant that helps the user verify whether an entity is captured in a list."
    prompt = """I will give you one entity and one list, and I want you to respond with \"YES\" if the entity is within the list, \"NO\" if it is not in the list.  
    
    List: 
    {}
    Entity: {}
    Give your response in the following format: 
    `Reference in the list: {{item in the list if exists, None otherwise}}
    Answer: {{YES or NO}}` and say nothing else.
    """
    response = completion_with_backoff(
      engine=engine,
      messages = [{"role":"system","content":system_prompt},
                  {"role":"user","content":prompt.format(list_in_txt, response)},],
      temperature=0,
      max_tokens=25,
      top_p=0.95,
      frequency_penalty=0,
      presence_penalty=0,
      stop=None)
    result = response["choices"][0]["message"]["content"]

    try:
        lines = result.split("\n")
        reference = lines[0].split(": ")[1]
        answer = lines[1].split(": ")[1]
        return reference, answer
    except:
        return None, None
    return result


In [ ]:
import pickle

def save_output(records, correctness, output_file):
    records_final = []

    for i in range(len(records['prompt'])):
        prompt = records['prompt'][i]
        completion = records['completion'][i]
        record = (prompt, completion, [correctness[i][0], correctness[i][1]])
        records_final.append(record)

    # Save records_final as a pkl file
    with open(output_file, 'wb') as file:
        pickle.dump(records_final, file)


In [ ]:
def remove_delimiter(name):
    delimiters = ['.', ',']
    for delimiter in delimiters:
        if name.endswith(delimiter):
            return name[:-len(delimiter)]
    return name


In [ ]:
def query_wikidata(person_name):
    url = 'https://query.wikidata.org/sparql'
    query = '''
       SELECT DISTINCT ?personLabel ?awardLabel ?cityLabel WHERE {{
       ?person rdfs:label "{0}"@en .
       ?person wdt:P166 ?award .
       ?person wdt:P19 ?city .
  
       SERVICE wikibase:label {{
          bd:serviceParam wikibase:language "[AUTO_LANGUAGE],en" .
       }}
    }}
    '''.format(person_name)
    
    try:
    
        response = requests.get(url, params={'format': 'json', 'query': query})
        data = response.json()
    
    except:
        print("wiki query failed")
        return None
    return data


def organize_data(data):
    organized_data = []
    
    if data is not None:
        for item in data['results']['bindings']:
            person = item['personLabel']['value']
            award = item['awardLabel']['value']
            city = item['cityLabel']['value']

            organized_data.append({
                "Person": person,
                "Award" : award,
                "City": city,
            })
    return organized_data

def get_nobel_info(person):

    nobel_winner_data = query_wikidata(person)
    nobel_winner_organized_data = organize_data(nobel_winner_data)

    matching_awards = []
    matching_cities = []
    for entry in nobel_winner_organized_data:
        if entry['Person'].lower() == person.lower():
            if entry['Award'] not in matching_awards:
                matching_awards.append(entry['Award'])
                
            if entry['City'] not in matching_cities:   
                matching_cities.append(entry['City'])
                
    return matching_awards, matching_cities
    

In [ ]:
import time

def verify_nobel_winner(records, debug_file):
    
    correctness = np.zeros((len(records["prompt"]), 2))
    
    with open(debug_file, "w") as fd:
        for i in range(len(records["prompt"])):
            time.sleep(5)
            completion = records["completion"][i]
            completion = completion.split("\n")[0]
            completion = remove_delimiter(completion).strip()       
            awards, cities = get_nobel_info(completion)
            fd.write(f"Completion : {completion}\n")
            fd.write(f"Awards retrieved from wiki : {awards}\n")
            fd.write(f"City retrieved from wiki : {cities}\n")

            for constraint in records["name"][i]:
                if "was born" in constraint.lower():                                      
                    city_reference, city_answer = verify_entity_exists(constraint, cities)            

                    fd.write(f"City constraint : {constraint}\n")
                    fd.write(f"Turbo City refernce : {city_reference}\n")
                    fd.write(f"Turbo City answer : {city_answer}\n")
                    fd.write(f"=====================================\n")
                    if city_answer is not None:
                        correctness[i][1] = (1 if city_answer.lower() == "yes" else 0)
                else:
                    award_reference, award_answer = verify_entity_exists(constraint, awards)            

                    fd.write(f"award constraint : {constraint}\n")
                    fd.write(f"Turbo award refernce : {award_reference}\n")
                    fd.write(f"Turbo award answer : {award_answer}\n")
                    fd.write(f"=====================================\n")
                    if award_answer is not None:
                        correctness[i][0] = (1 if award_answer.lower() == "yes" else 0)     

    return correctness

In [ ]:
import pickle
import os
import numpy as np
from easydict import EasyDict as edict

data_pretty = {
    "nobel_city": "Nobel Winners"
}
result_records = []
for model_size in ["7b", "13b", "70b"]:
    for data_name in data_pretty:
        filename = f"./outputs/Llama-2-{model_size}-hf_{data_name}_localized_track.pkl"
        output_file = f"./outputs/Llama-2-{model_size}-hf_{data_name}_localized_track.pkl"
        debug_file = f"./outputs/Llama-2-{model_size}-hf_{data_name}_localized_track.debug"

        if not os.path.exists(filename):
            print(filename)
            continue
        records_to_save = edict(pickle.load(open(filename, "rb")))
        records = records_to_save
        correctness = verify_nobel_winner(records, debug_file) 
        save_output(records, correctness, output_file )